# Data Assembly

In [1]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
import queries as q
import seaborn as sns
%config InlineBackend.figure_formats = set(['retina'])
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pyplot_themes as themes
import cats
from con import con
import re
import altair as alt
from missingno import missingno
from texter import send

In [2]:
# plot settings
%run -i 'eplots.py'

## Data Imports

In [17]:
# pull tables from postgresql
apts = pd.concat(pd.read_sql(q.apt_sql,con, chunksize = 500)) # apartment complexes and units
res = pd.concat(pd.read_sql(q.res_sql,con, chunksize = 500)) # residential buildings (<5 units)
condo = pd.concat(pd.read_sql(q.condo_sql,con, chunksize = 500)) # condominium complexes and units
mhpad = pd.concat(pd.read_sql(q.mh_sql,con, chunksize = 500)) # mobile home pads
lookup = pd.concat(pd.read_sql_table('lookup', con, chunksize = 500)) # category lookup tables 
pp = pd.concat(pd.read_sql(q.pp_sql,con, chunksize = 500)) # parcel-precincts
turnout = pd.concat(pd.read_sql(q.turnout_sql, con, chunksize = 500)) # historical precinct voter turnout
owner_residency = pd.concat(pd.read_sql_table('parcel_owner_residency', con, chunksize = 500)) # owner residency

In [18]:
# filter desired columns
apts = apts[cats.keep_apts]
res = res[cats.keep_res]
condo = condo[cats.keep_condo]
mhpad = mhpad[cats.keep_mhpad]

# add source table identifier
apts['apt'] = 1
res['res'] = 1
condo['condo'] = 1
mhpad['mh'] = 1

In [19]:
cc = cats.usedcols # columns with encoded variables
lu = {} # dictionary for replacing encoded variables

# build lookup table
lookup['luitem'] = lookup['luitem'].str.rstrip().astype(int)
lookup['lutype'] = lookup['lutype'].str.rstrip().astype(int)
lookup['ludescription'] = lookup['ludescription'].str.replace(r'[^\w\s]+', '').replace('  ','')
lookup['ludescription'] = lookup['ludescription'].str.strip()

for x in set(lookup['lutype']):
    l = lookup[lookup['lutype'] == x]
    lu[x] = pd.Series(l.ludescription.values,index=l.luitem.astype(str)).to_dict()
lu = { k : { str(x) : y for x,y in v.items() } for k,v in lu.items() }

# replace encoded categorical values
cats.rcats(res,'res',cc,lu)
cats.rcats(apts,'apt',cc,lu)
cats.rcats(condo,'condo',cc,lu)

-----
## Clean Housing Unit Tables  
*Different housing types are represented in different ways. They need to be cleaned to be combined.*

### Apartments

In [20]:
apt_rename = {
    'effyr' : 'yrrenovated',
    'nbrbedrooms' : 'beds',
    'nbrbaths' : 'baths',
    'nbrthistype' : 'units',
    'nbrstories' : 'stories',
    'sqft' : 'interior_area'
    }
apts.rename(columns=apt_rename, inplace=True)
apts['units'] = apts['units'].fillna(apts['nbrunits'])
apts['units'] = apts['units'].astype(int)
apts['units'].fillna(value=apts['nbrunits'], inplace=True)
apts[['apprlandval']].fillna(value=np.nan, inplace=True)
apts.reset_index(inplace=True)
apts = apts.loc[:, ~apts.columns.duplicated()]

# housing units per building
phu = apts.groupby(['major','minor'])['units'].agg([('hu', 'sum')]).reset_index()
apts = apts.merge(phu, left_on = ['major', 'minor'], right_on = ['major', 'minor'])
apts = apts[apts['units'] != 0]

In [21]:
# land value, improvement value, and land area per housing unit
apts['landperunit'] = apts['sqftlot'].astype(float) / apts['hu'].astype(float)
apts['landvalunit'] = apts['apprlandval'].astype(float) / apts['hu'].astype(float)
apts['impsvalunit'] = apts['apprimpsval'].astype(float) / apts['hu'].astype(float)

# missing data
apts['beds'].fillna(apts['avgunitsize'].apply(lambda x: (int(x) / 800)))
apts['baths'].fillna(1)
apts['interior_area'].fillna(apts['avgunitsize'].astype(int))

# building ages
apts['age'] = 2020 - apts['yrbuilt'].astype(int)
apts.loc[apts['yrrenovated'].astype(int) == 0, 'yrrenovated'] = apts['yrbuilt'].astype(int)
apts['effage'] = 2020 - apts['yrrenovated'].astype(int)

# repeat rows per housing unit
apts = apts.loc[apts.index.repeat(apts.units)]

# drop processed columns
apts.drop(columns=['yrrenovated', 'yrbuilt'], inplace=True)

In [22]:
apts

,index,unittypeitemid,units,interior_area,beds,baths,major,minor,nbrbldgs,stories,nbrunits,avgunitsize,projectlocation,projectappeal,pcntwithview,constrclass,bldgquality,condition,elevators,sectysystem,fireplace,laundry,addrline,citystate,rpzip,taxvalreason,apprlandval,apprimpsval,proptype,districtname,currentzoning,presentuse,sqftlot,watersystem,sewersystem,access,topography,streetsurface,inadequateparking,mtrainier,olympics,cascades,territorial,seattleskyline,pugetsound,lakewashington,lakesammamish,smalllakerivercreek,otherview,wfntlocation,wfntfootage,trafficnoise,powerlines,othernuisances,adjacentgolffairway,adjacentgreenbelt,apt,hu,landperunit,landvalunit,impsvalunit,age,effage
0,0,Flat,2,827,2,1,010800,0020,2,2,4,856,AVERAGE,AVERAGE,0,WOOD FRAME,AVERAGE,Good,N,N,N,PRIVATE,PO BOX 206,FALL CITY WA,98024,,1571400,1000,C,SEATTLE,LR2 (M),4Plex,14966,WATER DISTRICT,PUBLIC,PUBLIC,0,PAVED,2,0,0,0,0,0,0,0,0,0,0,0,0,0,N,N,N,N,1,4,3741.5000000,392850.0000000,250.0000000,56,24
0,0,Flat,2,827,2,1,010800,0020,2,2,4,856,AVERAGE,AVERAGE,0,WOOD FRAME,AVERAGE,Good,N,N,N,PRIVATE,PO BOX 206,FALL CITY WA,98024,,1571400,1000,C,SEATTLE,LR2 (M),4Plex,14966,WATER DISTRICT,PUBLIC,PUBLIC,0,PAVED,2,0,0,0,0,0,0,0,0,0,0,0,0,0,N,N,N,N,1,4,3741.5000000,392850.0000000,250.0000000,56,24
1,1,Townhouse,1,1100,2,1.75,010800,0020,2,2,4,856,AVERAGE,AVERAGE,0,WOOD FRAME,AVERAGE,Good,N,N,N,PRIVATE,PO BOX 206,FALL CITY WA,98024,,1571400,1000,C,SEATTLE,LR2 (M),4Plex,14966,WATER DISTRICT,PUBLIC,PUBLIC,0,PAVED,2,0,0,0,0,0,0,0,0,0,0,0,0,0,N,N,N,N,1,4,3741.5000000,392850.0000000,250.0000000,56,24
2,2,Flat,1,672,1,1,010800,0020,2,2,4,856,AVERAGE,AVERAGE,0,WOOD FRAME,AVERAGE,Good,N,N,N,PRIVATE,PO BOX 206,FALL CITY WA,98024,,1571400,1000,C,SEATTLE,LR2 (M),4Plex,14966,WATER DISTRICT,PUBLIC,PUBLIC,0,PAVED,2,0,0,0,0,0,0,0,0,0,0,0,0,0,N,N,N,N,1,4,3741.5000000,392850.0000000,250.0000000,56,24
3,3,Flat,18,850,2,1,011410,0902,5,3,75,793,AVERAGE,AVERAGE,0,WOOD FRAME,AVERAGE,Average,N,N,Y,PRIVATE,9212 NE 20TH ST,CLYDE HILL WA,98004,,2222900,14746500,C,KENMORE,R24,Apartment,105857,WATER DISTRICT,PUBLIC,PUBLIC,0,PAVED,2,0,0,0,0,0,0,0,0,0,0,0,0,0,N,N,N,N,1,75,1411.4266667,29638.6666667,196620.0000000,32,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24976,476,Flat,6,934,2,1,982870,2435,1,3,6,934,AVERAGE,AVERAGE,50,WOOD FRAME,AVERAGE,Good,N,Y,N,0,2110 WESTERN AVE,SEATTLE WA,98121,,690000,1847000,C,SEATTLE,LR3 (M),Apartment,3000,WATER DISTRICT,PUBLIC,PUBLIC,0,0,2,0,0,AVERAGE,AVERAGE,0,0,0,0,0,0,0,0,MODERATE,N,N,N,N,1,6,500.0000000,115000.0000000,307833.3333333,111,26
24976,476,Flat,6,934,2,1,982870,2435,1,3,6,934,AVERAGE,AVERAGE,50,WOOD FRAME,AVERAGE,Good,N,Y,N,0,2110 WESTERN AVE,SEATTLE WA,98121,,690000,1847000,C,SEATTLE,LR3 (M),Apartment,3000,WATER DISTRICT,PUBLIC,PUBLIC,0,0,2,0,0,AVERAGE,AVERAGE,0,0,0,0,0,0,0,0,MODERATE,N,N,N,N,1,6,500.0000000,115000.0000000,307833.3333333,111,26
24976,476,Flat,6,934,2,1,982870,2435,1,3,6,934,AVERAGE,AVERAGE,50,WOOD FRAME,AVERAGE,Good,N,Y,N,0,2110 WESTERN AVE,SEATTLE WA,98121,,690000,1847000,C,SEATTLE,LR3 (M),Apartment,3000,WATER DISTRICT,PUBLIC,PUBLIC,0,0,2,0,0,AVERAGE,AVERAGE,0,0,0,0,0,0,0,0,MODERATE,N,N,N,N,1,6,500.0000000,115000.0000000,307833.3333333,111,26
24976,476,Flat,6,934,2,1,982870,2435,1,3,6,934,AVERAGE,AVERAGE,50,WOOD FRAME,AVERAGE,Good,N,Y,N,0,2110 WESTERN AVE,SEATTLE WA,98121,,690000,1847000,C,SEATTLE,LR3 (M),Apartment,3000,WATER DISTRICT,PUBLIC,PUBLIC,0,0,2,0,0,AVERAGE,AVERAGE,0,0,0,0,0,0,0,0,MODERATE,N,N,N,N,1,6,500.0000000,115000.0000000,307833.3333333,111,26


### Residential Buildings

In [23]:
res_rename = {
    'nbrlivingunits' : 'units',
    'sqfttotliving' : 'interior_area'
}

res.rename(columns = res_rename, inplace=True)
# res.dropna(subset=['units','apprimpsval'], inplace=True)
res.reset_index(inplace=True)

In [24]:
# housing units per building/parcel
res['units'] = res['units'].astype(int)
phu = res.groupby(['major','minor'])['units'].agg([('hu', 'sum')]).reset_index()
res = res.merge(phu, left_on = ['major', 'minor'], right_on = ['major', 'minor'])

# land value, improvement value, and land area per housing unit
res['landperunit'] = res['sqftlot'].astype(float) / res['hu']
res['landvalunit'] = res['apprlandval'].astype(float) / res['hu']
res['impsvalunit'] = res['apprimpsval'].astype(float) / res['hu']

# building ages
res['age'] = 2020 - res['yrbuilt'].astype(int)
res.loc[res['yrrenovated'].astype(int) == 0, 'yrrenovated'] = res['yrbuilt'].astype(int)
res['effage'] = 2020 - res['yrrenovated'].astype(int)

# repeat rows per housing unit
res = res.loc[res.index.repeat(res.units)]
res['beds'] = res['bedrooms'].astype(float) / res['hu']
res['baths'] = (res['bathfullcount'].astype(float) + res['bath3qtrcount'].astype(float) + res['bathhalfcount'].astype(float)) / res['hu']

# drop processed columns
res.drop(columns = ['bathfullcount', 'bath3qtrcount', 'bathhalfcount', 'bedrooms', 'yrbuilt', 'yrrenovated'], inplace=True)

### Condominia

In [25]:
condo_rename = {
    'nbrlivingunits' : 'units',
    'nbrbedrooms' : 'beds',
    'nbrstories' : 'stories',
    'footage' : 'interior_area'
}

condo.rename(columns = condo_rename, inplace=True)
# condo.dropna(subset=['apprimpsval'], inplace=True)
condo.reset_index(inplace=True)
condo = condo.loc[:, ~condo.columns.duplicated()]
condo['hu'] = condo.groupby('major')['major'].transform('count')
condo['beds'] = condo['beds'].replace(['S', ' '], ['0', '0'])

# land value, improvement value, and land area per housing unit
condo['landperunit'] = condo['sqftlot'].astype(float) / condo['hu']
condo['landvalunit'] = condo['apprlandval'].astype(float)
condo['impsvalunit'] = condo['apprimpsval'].astype(float)

# bathrooms
condo['baths'] = (condo['bathfullcount'].astype(float) + condo['bath3qtrcount'].astype(float) + condo['bathhalfcount'].astype(float))

# building ages
condo['age'] = 2020 - condo['yrbuilt'].astype(int)
condo.loc[condo['effyr'].astype(int) == 0, 'effyr'] = condo['yrbuilt'].astype(int)
condo['effage'] = 2020 - condo['effyr'].astype(int)

# drop processed columns
condo.drop(columns = ['bathfullcount', 'bath3qtrcount', 'bathhalfcount', 'apprlandval', 'apprimpsval', 'yrbuilt', 'effyr'], inplace=True)

-----
### Mobile Home Pads

In [26]:
# impute missing ages
mhpad.loc[(mhpad['effyr'] == '0'),'effyr'] = mhpad.loc[mhpad['effyr'] != '0','effyr'].astype(float).mean()

# pad ages
mhpad['age'] = 2020 - mhpad['effyr'].astype(int)
mhpad['effage'] = mhpad['age']

# bed and bath
mhpad['beds'] = 1
mhpad['baths'] = 1

# other
mhpad['interior_area'] = 1000
mhpad['stories'] = 1


# land value, improvement value, and land area per housing unit
mhpad['impsvalunit'] = mhpad['accyvalue'].astype(int) / mhpad['quantity'].astype(int)
mhpad['landvalunit'] = mhpad['apprlandval'].astype(int) / mhpad['quantity'].astype(int)
mhpad['landperunit'] = mhpad['sqftlot'].astype(int) / mhpad['quantity'].astype(int)

# drop processed columns
mhpad['mh'] = 1

_____
## Concatenate Housing Units

In [27]:
homes = pd.concat([apts.loc[:,~apts.columns.duplicated()],res.loc[:,~res.columns.duplicated()],condo.loc[:,~condo.columns.duplicated()],mhpad.loc[:,~mhpad.columns.duplicated()]])

In [28]:
# clean some columns
homes['res'].fillna(value = 0, inplace=True)
homes['apt'].fillna(value = 0, inplace=True)
homes['condo'].fillna(value = 0, inplace=True)
homes['mh'].fillna(value = 0, inplace=True)
homes['topography'] = homes['topography'].str.replace('YES','1').astype(int)
homes['sectysystem'].replace(' ', 1, inplace=True)
homes['fireplace'].replace(' ', 1, inplace=True)
homes['elevators'].replace(' ', 1, inplace=True)

In [29]:
# create new housing type categories
homes['single_family'] = (homes['res']==1).astype(int) * (homes['units']==1).astype(int)
homes['duplex'] = homes['presentuse'].isin(['Duplex']).astype(int)
homes['triplex'] = homes['presentuse'].isin(['Triplex']).astype(int)
homes['fourplex'] = homes['presentuse'].isin(['4Plex']).astype(int)
homes['townhouse'] = homes['presentuse'].isin(['Townhouse Plat']).astype(int)
homes['apartment'] = ((homes['presentuse'].isin(['Apartment','ApartmentCoop','ApartmentMixed Use','ApartmentSubsidized'])) | (homes['apt'] == 1)).astype(int)
homes['senior_housing'] = homes['presentuse'].isin(['Retirement Facility','Nursing Home']).astype(int)
homes['student_housing'] = homes['presentuse'].isin(['FraternitySorority House','ResidenceHallDorm']).astype(int)
homes['apt_subsidized'] = homes['presentuse'].isin(['ApartmentSubsidized']).astype(int)
homes['mobile_home'] = homes['presentuse'].isin(['CondominiumM Home Pk','Mobile Home', '']).astype(int)
homes['condominium'] = homes['presentuse'].isin(['CondominiumResidential','CondominiumMixed Use']).astype(int)
homes['mixed_use_mf'] = homes['presentuse'].isin(['ApartmentMixed Use','CondominiumMixed Use']).astype(int)

In [33]:
# filter by parcel use
homes = homes[homes['presentuse'].isin(cats.uses)]

# replace N/Y
homes.replace(['N','Y'],[0,0], inplace=True)

# add PIN
homes['pin'] = homes['major'] + homes['minor']

In [34]:
len(homes)

987288

In [35]:
homes['sectysystem'].replace(' ', 1, inplace=True)
homes['fireplace'].replace(' ', 1, inplace=True)
homes['elevators'].replace(' ', 1, inplace=True)

In [37]:
homes.loc[(homes['interior_area'] == 0) & (homes['avgunitsize'] != 0),'interior_area'] = homes.loc[(homes['interior_area'] == 0) & (homes['avgunitsize'] != 0),'avgunitsize']

In [38]:
#homes.loc[(homes['sqfttotliving'] != 0) & (homes['interior_area'] == 0),'interior_area'] = # homes.loc[(homes['sqfttotliving'] != 0) & (homes['interior_area'] == 0),'sqfttotliving']
homes['garagesize'] = homes['sqftgaragebasement'].astype(float) + homes['sqftgarageattached'].astype(float)
homes = homes.replace(r'^\s*$', np.nan, regex=True)

In [39]:
homes['beds'] = homes['beds'].str.strip().replace(['S',''],['.25',1]).astype(float)

In [40]:
homes.fillna(0, inplace=True)

In [41]:
homes.to_pickle('homes')

In [42]:
len(homes)

987288

## Combine with Voter Registrations by Address

In [43]:
voters = pd.read_pickle('voters')

In [44]:
# aggregate condominium major parcels
cpins = pd.Series(list(set(homes[homes['complextype'] != 0]['major'])))
cpins.to_csv('cpins.csv')
# combine condo parcels
noncon = homes[homes['complextype'] == 0].merge(voters[~voters['MAJOR'].isin(cpins)], how='left', left_on=['major','minor'], right_on=['MAJOR','MINOR'])
con = homes[homes['complextype'] != 0].merge(voters[voters['MAJOR'].isin(cpins)], how='left', left_on=['major'], right_on=['MAJOR'])
homes = pd.concat([con,noncon])

In [49]:
owner_residency = owner_residency.groupby(['major','minor']).agg('first').reset_index()

In [50]:
homes = pd.merge(homes,owner_residency, how='left', left_on=['major','minor'], right_on=['major','minor'])

In [51]:
homes['voters'].fillna(0,inplace=True)
homes['vpu'] = homes['voters'] / homes['hu']
homes['v19pu'] = homes['voted19'] / homes['hu']
homes['v20pu'] = homes['voted20'] / homes['hu']

In [53]:
homes.fillna(0, inplace=True)

In [54]:
# list of columns we don't need
drops = ['index_x','addrline','citystate','rpzip','MAJOR','MINOR','index_y','pcntnetcondition','accyvalue','datevalued','bldggradevar','bldgnbr','sqftlot','rpzip','major','minor', 'nbrunits','sqftgaragebasement','sqftgarageattached','currentzoning','presentuse','res','condo','mh','apt','voters','gender','hu','apprlandval','apprimpsval','voted19','voted20']
homes.drop(columns=drops, inplace=True)

In [55]:
homes.to_pickle('home_votes')

________________
## Voter Turnout Precinct Data

In [ ]:
pm = pp.merge(turnout,how='outer',left_on=['name','year'],right_on=['name','y'])
pm['pin'] = pm['major'] + pm['minor']
pm = pm[['pin','voters','ballots','et']]
pm = pm.groupby(by=['pin','et'],as_index=False).sum()
pm['turnout'] = pm['ballots'] / pm['voters']
pm = pm.pivot(index='pin', columns='et', values=['turnout'])
pm.columns = pm.columns.get_level_values(1)

In [ ]:
pm.reset_index(inplace=True)

In [ ]:
pm.to_csv('p_to.csv')

In [ ]:
send('Done with home_votes')